In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import sys
import os
import glob
import numpy as np
import pandas as pd
import dask.dataframe as dd
import xarray as xr
from dask import delayed, compute
import itertools
from pytmatrix import tmatrix_aux, refractive, tmatrix, radar
from pymiecoated import Mie
from scipy.constants import c
from sqlalchemy.exc import OperationalError
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from re import split
import matplotlib.dates as mdates
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, progress
import dask.dataframe as dd
from dask import delayed, compute
from metpy.calc import pressure_to_height_std as p2h
from metpy.calc import lcl

import cartopy.crs as ccrs
import cartopy
import cartopy.mpl.ticker as cticker
import cartopy.io.img_tiles as cimgt
import cartopy.feature as cfeature
import warnings
warnings.filterwarnings('ignore')

sys.path.insert(1, f"{os.path.abspath(os.path.join(os.path.abspath(''), '../'))}")
from src.utils import get_pars_from_ini, make_dir

location = split(', |_|-|!', os.popen('hostname').read())[0].replace("\n", "")
path_data = get_pars_from_ini(file_name='loc')[location]['path_data']

In [ ]:
import cython
%load_ext cython

In [ ]:
%%cython 
import numpy as np
cimport numpy as np

cdef double psd_mm_m(double psd):
    return psd * 1e6

cdef double d_mm(double d):
    return d * 1e-3

cpdef double[:, ::1] comp_tiles_cython(double[:, ::1] psd, double[:] diam, double[:] d_d):
    cdef:
        double d_tile, dd_tile,  _psd, _d, _dd
        Py_ssize_t xmax = psd.shape[0]
        Py_ssize_t ymax = psd.shape[1]
        double [:, ::1] result = np.zeros((xmax, ymax)) 
        double [:, ::1] lwc  = result
        int i, j
    for i in range(xmax):
        for j in range(ymax):
            _psd = psd_mm_m(psd[i, j])
            _d = d_mm(diam[j])
            _dd = d_mm(d_d[j])
            lwc[i, j] = (3.1416 / (6.  * 1000.)) * (_psd) * (_d) ** 3 * (_dd)
    return lwc

In [ ]:
def comp_lwc(t, p):
    """
    """
    g = 9.8076
    cp = 1004.0
    Rd = 287.0
    Rv = 461.5
    # t = 273.15 + 8.0
    # p = 900.0
    L = 2.5e6
    es = 6.112 * np.exp(17.67 * (t - 273.15) / (243.5 + (t - 273.15)))
    molar_mass_ratio = 1.0 / 1.608  # Rd/Rv
    r = molar_mass_ratio * es / p
    gamma_d = -g / cp
    gamma_m = gamma_d * (1.0 + L * r / (Rd * t)) / (1 + L ** 2 * r / (cp * Rv * t ** 2))
    rho_a = 100.0 * p / (Rd * t)  # hPa back to Pa for density calculation
    cw = rho_a * (cp / L) * (gamma_m - gamma_d)  # 1e3 for units. End result in g/m^3
    return cw

def comp_tiles(xr_dset):
    try:
        d_tile = np.tile(xr_dset.diameter, [np.shape(xr_dset.psd.values)[0], 1])
        dd_tile = np.tile(xr_dset.d_d, [np.shape(xr_dset.psd.values)[0], 1])
        return d_tile, dd_tile
    except ValueError:
        d_tile = np.tile(xr_dset.diameter, [np.shape(xr_dset.psd[:, :, 0].values), 1])
        dd_tile = np.tile(xr_dset.d_d, [np.shape(xr_dset.psd[:,:, 0].values), 1])
        return d_tile, dd_tile
    
def comp_lwc_raw(xr_dset):
    d, d_d = comp_tiles(xr_dset) 
    try:
        return (np.pi / (6.  * 1000.)) * (xr_dset.psd * 1e6) * (d * 1e-3) ** 3 * (d_d * 1e-3)
    except ValueError:
        return (np.pi / (6.  * 1000.)) * (xr_dset.psd[:, :, 0] * 1e6) * (d * 1e-3) ** 3 * (d_d * 1e-3)
        
def add_colorbar_outside(im, ax, label):
    fig = ax.get_figure()
    bbox = ax.get_position() #bbox contains the [x0 (left), y0 (bottom), x1 (right), y1 (top)] of the axis.
    width = 0.01
    eps = 0.05 #margin between plot and colorbar
    # [left most position, bottom position, width, height] of color bar.
    cax = fig.add_axes([bbox.x1 + eps, bbox.y0, width, bbox.height])
    cbar = fig.colorbar(im, cax=cax, label=label)
    

In [ ]:
cdict = {'red': ((0., 1, 1),
                 (0.05, 1, 1),
                 (0.11, 0, 0),
                 (0.66, 1, 1),
                 (0.89, 1, 1),
                 (1, 0.5, 0.5)),
         'green': ((0., 1, 1),
                   (0.05, 1, 1),
                   (0.11, 0, 0),
                   (0.375, 1, 1),
                   (0.64, 1, 1),
                   (0.91, 0, 0),
                   (1, 0, 0)),
         'blue': ((0., 1, 1),
                  (0.05, 1, 1),
                  (0.11, 1, 1),
                  (0.34, 1, 1),
                  (0.65, 0, 0),
                  (1, 0, 0))}

my_cmap = LinearSegmentedColormap('my_colormap', cdict, 256)

In [ ]:
cluster = SLURMCluster(queue="seseml",
                       memory='10GB',
                       cores=40,
                       processes=1,
                       walltime='24:00:00',
                       scheduler_options={'host': '172.22.179.3:7223', 'dashboard_address': ':7799'})

cluster.scale(jobs=4)
cluster

In [ ]:
client = Client(cluster)
client

In [ ]:
%%bash
squeue -u $USER

In [ ]:
# # rdm_idx = pd.date_range(start='2019-09-06 23:58:30', periods=60, tz='UTC', freq='S')  # for P3B
# # rdm_idx = pd.date_range(start='2019-09-07 2:31:45', periods=150, tz='UTC', freq='S')  # for Lear
# start = '2019-09-07 02:31:45'
# end = '2019-09-07 02:33:30'
aircraft = 'Lear'
aircraft2 = 'Learjet'

In [ ]:
store1 = f"{path_data}/zarr/combined_psd_{aircraft}_20_800_FCDP_2DS_HVPS.zarr"
store2 = f"{path_data}/zarr/combined_psd_{aircraft}_800_1200.zarr"
store3 = f"{path_data}/zarr/2DS10_{aircraft2}.zarr"
store4 = f"{path_data}/zarr/HVPS_{aircraft2}.zarr"
store5 = f"{path_data}/zarr/Page0_Learjet.zarr"
store6 = f"{path_data}/zarr/FFSSP_{aircraft2}.zarr"
store7 = f"{path_data}/zarr/FCDP_{aircraft2}.zarr"

In [ ]:
xr_comb_350 = xr.open_zarr(store1)
xr_comb_800 = xr.open_zarr(store2)
xr_2ds = xr.open_zarr(store3)#.resample(time='5S').mean()
xr_hvps = xr.open_zarr(store4)#.resample(time='5S').mean()
xr_fssp = xr.open_zarr(store6)#.resample(time='5S').mean()
xr_fcdp = xr.open_zarr(store7)#.resample(time='5S').mean()
xr_add = xr.open_zarr(store5)#.resample(time='5S').mean()

In [ ]:
for _idx, data in xr_2ds.groupby('time.day'):

    fig, ax1 = plt.subplots(figsize=(12, 3))
    cbar = ax1.pcolormesh(data.time.values, data.diameter / 1e3, 
                   np.log10(data.psd.T.values * 1e6), 
                      vmin=0, vmax=10, cmap=my_cmap)
    ax1.set_yscale('log')
    lwc = comp_lwc_raw(data)
    lwc = lwc.where(lwc >0.1, 0)
    ax11 = ax1.twinx()
    ax11.plot(lwc.time, lwc.sum('diameter'))
    add_colorbar_outside(cbar, ax1, 'Log10 (N(D))')
    ax1.set_xlabel('Time (UTC)')
    ax1.set_ylabel('Diameter (mm)')
    ax11.set_ylabel('LWC (g m-3)')
    title = f"{pd.to_datetime(data.time.values[0]): %Y-%m-%d} UTC - {data.attrs['aircraft']}"
    fig.suptitle(title, fontsize=14, fontweight='bold', y=0.99)


In [ ]:
lwc_nev = xr_add.NevLWC.sel(time=slice('2019-09-07 02:31:30', '2019-09-07 02:34:10'))

In [ ]:
data_1 = xr_2ds.sel(time=slice('2019-09-07 02:31:30', '2019-09-07 02:34:10'))

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6), dpi=300, sharex=True)

cbar = ax1.pcolormesh(data_1.time.values, 
                      data_1.diameter / 1e3, 
                      np.log10(data_1.psd.T.values * 1e6), 
                      vmin=0, vmax=10, cmap=my_cmap)

ax1.set_yscale('log')

lwc_1 = comp_lwc_raw(data_1)

# ax11 = ax1.twinx()
# ax11.plot(lwc_1.time, lwc_1.sum('diameter'), c='k', lw=0.5)
add_colorbar_outside(cbar, ax1, label='Log10(N(D)) (Log10(mm-1 m-3))')

cbar2 = ax2.pcolormesh(lwc_1.time.values, 
                       lwc_1.diameter / 1e3, 
                       lwc_1.T.values, 
                       vmin=0, vmax=1, cmap=my_cmap)
ax22 = ax2.twinx()
ax22.plot(lwc_1.time, lwc_1.sum('diameter'), c='k', ls='--', lw=0.8, label="LWC 2DS")
ax22.plot(lwc_nev.time, lwc_nev, c='r', ls='--', lw=0.8, label='Nevzorov')



ax2.set_yscale('log')
ax2.set_ylim(0.01, 10)
add_colorbar_outside(cbar2, ax2, 'LWC (g m-3 mm)')
ax1.set_ylabel('Diameter (mm)')
ax2.set_ylabel('Diameter (mm)')
ax2.set_xlabel('Time (UTC)')
ax22.set_ylabel('LWC (g m-3)')
ax22.legend()
ax11.set_ylabel('LWC (g m-3)')
ax1.set_ylim(0.002, 10)
ax2.set_ylim(0.002, 10)
title = f"{pd.to_datetime(data_1.time.values[0]): %Y-%m-%d} UTC - {data.attrs['aircraft']}"
fig.suptitle(title, fontsize=14, fontweight='bold', y=0.99)
path_save = f"{path_data}/results/qc_cloud_probes/{xr_2ds.attrs['aircraft']}/{xr_2ds.attrs['instrument']}"
make_dir(path_save)
fig.savefig(f"{path_save}/{pd.to_datetime(data_1.time.values[0]): %Y-%m-%d}_to_{pd.to_datetime(data_1.time.values[-1]): %Y-%m-%d}.jpg")
# fig.tight_layout()

In [ ]:
alt = xr_add.Palt.sel(time=slice('2019-09-07 02:31:30', '2019-09-07 02:34:10')) * 0.3048

In [ ]:
ds = xr.open_dataset('/data/keeling/a/alfonso8/gpm/camp2ex/data/dropsondes/CAMP2EX_AVAPS_RD41_v1_20190907_030551.nc')
# ds = xr.open_dataset('C:/Users/alfonso8/Downloads/CAMP2EX_AVAPS_RD41_v1_20190907_032811.nc')
p = ds.dropna('time')['pres']
t = ds.dropna('time')['tdry']
dp = ds.dropna('time')['dp']
lcl_p, lcl_t = lcl(p, t, dp)
cbh = p2h(lcl_p[0])
ds = ds.where(ds.pres < 930)
p = ds['pres'].dropna('time')
t = ds['tdry'].dropna('time')
dp = ds['dp'].dropna('time')
h = p2h(p)
lwc = comp_lwc(t.values + 273.15, p.values)
lwc_sum = np.cumsum(lwc[:-1] * h.diff('time').values * 1e6)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6), dpi=150)
ax.scatter(lwc_1.sum('diameter'), alt, edgecolors='k', marker='*', label='LWC 2DS', 
           facecolors='none')
ax.scatter(lwc_nev, alt, edgecolors='b', marker='s', facecolors='none', label='Nevzorov')
ax.set_xlabel('LWC (g m-3)')
ax.set_ylabel('Height (m)')
ax.legend()
ax.plot(lwc_sum, h[:-1] * 1000)
title = f"{pd.to_datetime(data_1.time.values[0]): %Y-%m-%d} UTC - {data_1.attrs['aircraft']}"
fig.suptitle(title, fontsize=14, fontweight='bold', y=0.99)
make_dir(path_save)
fig.savefig(f"{path_save}/{pd.to_datetime(data_1.time.values[0]): %Y-%m-%d}_to_{pd.to_datetime(data_1.time.values[-1]): %Y-%m-%d}_ALWC")

In [ ]:
data_1_filt = data_1.sel(diameter=slice(10, 1000))
lwc_filt = comp_lwc_raw(data_1_filt)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6), dpi=300, sharex=True)

cbar = ax1.pcolormesh(data_1_filt.time.values, 
                      data_1_filt.diameter / 1e3, 
                      np.log10(data_1_filt.psd.T.values * 1e6), 
                      vmin=0, vmax=10, cmap=my_cmap)

ax1.set_yscale('log')

# ax11 = ax1.twinx()
# ax11.plot(lwc_filt.time, lwc_filt.sum('diameter'), c='k', lw=0.5)
add_colorbar_outside(cbar, ax1, label='Log10(N(D)) (Log10(mm-1 m-3))')

cbar2 = ax2.pcolormesh(lwc_filt.time.values, 
                       lwc_filt.diameter / 1e3, 
                       lwc_filt.T.values, 
                       vmin=0, vmax=1, cmap=my_cmap)
ax22 = ax2.twinx()
ax22.plot(lwc_filt.time, lwc_filt.sum('diameter'), c='k', ls='--', lw=0.8, label="LWC 2DS")
ax22.plot(lwc_nev.time, lwc_nev, c='r', ls='--', lw=0.8, label='Nevzorov')



ax2.set_yscale('log')
ax2.set_ylim(0.01, 10)
add_colorbar_outside(cbar2, ax2, 'LWC (g m-3 mm)')
ax1.set_ylabel('Diameter (mm)')
ax2.set_ylabel('Diameter (mm)')
ax2.set_xlabel('Time (UTC)')
ax22.set_ylabel('LWC (g m-3)')
ax22.legend()
ax11.set_ylabel('LWC (g m-3)')

ax1.set_ylim(0.002, 10)
ax2.set_ylim(0.002, 10)

title = f"{pd.to_datetime(data_1_filt.time.values[0]): %Y-%m-%d} UTC - {data_1_filt.attrs['aircraft']}"
fig.suptitle(title, fontsize=14, fontweight='bold', y=0.99)
path_save = f"{path_data}/results/qc_cloud_probes/{xr_2ds.attrs['aircraft']}/{xr_2ds.attrs['instrument']}"
make_dir(path_save)
fig.savefig(f"{path_save}/{pd.to_datetime(data_1_filt.time.values[0]): %Y-%m-%d}_to_{pd.to_datetime(data_1_filt.time.values[-1]): %Y-%m-%d}_filt.jpg")
# fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(6,6), dpi=150)
ax.scatter(lwc_filt.sum('diameter'), alt, edgecolors='k', marker='*', label='LWC 2DS (filt)', 
           facecolors='none')
ax.scatter(lwc_nev, alt, edgecolors='b', marker='s', facecolors='none', label='Nevzorov')
ax.set_xlabel('LWC (g m-3)')
ax.set_ylabel('Height (m)')
ax.legend()
ax.plot(lwc_sum, h[:-1] * 1000)
title = f"{pd.to_datetime(data.time.values[0]): %Y-%m-%d} UTC - {data.attrs['aircraft']}"
fig.suptitle(title, fontsize=14, fontweight='bold', y=0.99)
make_dir(path_save)
fig.savefig(f"{path_save}/{pd.to_datetime(data_1_filt.time.values[0]): %Y-%m-%d}_to_{pd.to_datetime(data_1_filt.time.values[-1]): %Y-%m-%d}_ALWC_filt.jpg")

In [ ]:
lat = xr_add.Lat.sel(time='2019-09-07 02:33:00')
image_extent = [119, 120, 15.5, 16.5]

In [ ]:
lat = xr_add.Lat.sel(time=slice('2019-09-07 02:31:30', '2019-09-07 02:34:10'))
lon = xr_add.Long.sel(time=slice('2019-09-07 02:31:30', '2019-09-07 02:34:10'))
alt = xr_add.Palt.sel(time=slice('2019-09-07 02:31:30', '2019-09-07 02:34:10')) * 0.3048

lat_ds = ds.lat.min().values
lon_ds = ds.lon.min().values

In [ ]:
lat = xr_add.Lat.sel(time='2019-09-07 02:32:06')
lon = xr_add.Long.sel(time='2019-09-07 02:32:06')
alt = xr_add.Palt.sel(time='2019-09-07 02:32:02') * 0.3048

In [ ]:
min_lat = lat.min()
max_lat = lat.max()
min_lon = lon.min()
max_lon = lon.max()
image_extent = [118, 121, 15, 18]

In [ ]:
plt.rcParams.update({'font.size': 14})
axis_font = {'size':'15'}

fig = plt.figure(figsize=(15,12), dpi=150)
request = cimgt.GoogleTiles()
projection = ccrs.PlateCarree()
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent(image_extent)
ax.coastlines()
ax.add_image(request, 8)
ax.gridlines(draw_labels=True, rasterized=True)
ax.plot(lon, lat,  marker='o', markersize=7.0, markeredgewidth=2.5,
                 markerfacecolor='black', markeredgecolor='white',
                  transform=ccrs.PlateCarree())
at_x, at_y = ax.projection.transform_point(lon, lat, src_crs=ccrs.PlateCarree())
ax.annotate(
            '2DS - Leatrjet at 2:32:06 Z', xy=(at_x, at_y), xytext=(30, 20), textcoords='offset points',
            color='black', backgroundcolor='white', size='large',
            arrowprops=dict(arrowstyle='->', color='white', linewidth=2.5))


ax.plot(lon_ds, lat_ds,  marker='o', markersize=7.0, markeredgewidth=2.5,
                 markerfacecolor='black', markeredgecolor='white',
                  transform=ccrs.PlateCarree())
at_x, at_y = ax.projection.transform_point(lon_ds, lat_ds, src_crs=ccrs.PlateCarree())
ax.annotate(
            'Dropsonde at 3:05:51 Z', xy=(at_x, at_y), xytext=(30, 20), textcoords='offset points',
            color='black', backgroundcolor='white', size='large',
            arrowprops=dict(arrowstyle='->', color='white', linewidth=2.5))

# clb = plt.colorbar(bmap, aspect=40)
# ax.set_label('NASA P-3 aircraft height (km)', **axis_font)
make_dir(path_save)
fig.savefig(f"{path_save}/2ds_dropsonde_map.jpg")

In [ ]:
def bcksct(ds, instrument, ar=1, j=0) -> pd.DataFrame:
    """

    :param ds: numpy array of particle diameters. should be in millimeters
    :param ar: axis ratio of the particle
    :param j: Zenith angle input
    :return:
    """
    x_ku = 2 * np.pi * (ds / 2.) / tmatrix_aux.wl_Ku
    x_ka = 2 * np.pi * (ds / 2.) / tmatrix_aux.wl_Ka
    x_w = 2 * np.pi * (ds / 2.) / tmatrix_aux.wl_W
    # Tmatrix calculations
    tmat_ku = [radar.radar_xsect(tmatrix.Scatterer(radius=i / 2., wavelength=tmatrix_aux.wl_Ku,
                                                   m=refractive.m_w_0C[tmatrix_aux.wl_Ku], axis_ratio=1.0 / ar, thet0=j,
                                                   thet=180 - j,
                                                   phi0=0., phi=180., radius_type=tmatrix.Scatterer.RADIUS_MAXIMUM)) for
               i in ds]
    tmat_ka = [radar.radar_xsect(tmatrix.Scatterer(radius=i / 2., wavelength=tmatrix_aux.wl_Ka,
                                                   m=refractive.m_w_0C[tmatrix_aux.wl_Ka], axis_ratio=1.0 / ar, thet0=j,
                                                   thet=180 - j,
                                                   phi0=0., phi=180., radius_type=tmatrix.Scatterer.RADIUS_MAXIMUM)) for
               i in ds]
    tmat_w = [radar.radar_xsect(tmatrix.Scatterer(radius=i / 2., wavelength=tmatrix_aux.wl_W,
                                                  m=refractive.m_w_0C[tmatrix_aux.wl_W], axis_ratio=1.0 / ar, thet0=j,
                                                  thet=180 - j,
                                                  phi0=0., phi=180., radius_type=tmatrix.Scatterer.RADIUS_MAXIMUM)) for
              i in ds]

    # Mie calculations
    mie_ku = [Mie(x=x_ku[w], m=refractive.m_w_0C[tmatrix_aux.wl_Ku]).qb() * np.pi * (i / 2.) ** 2 for w, i in
              enumerate(ds)]
    mie_ka = [Mie(x=x_ka[w], m=refractive.m_w_0C[tmatrix_aux.wl_Ka]).qb() * np.pi * (i / 2.) ** 2 for w, i in
              enumerate(ds)]
    mie_w = [Mie(x=x_w[w], m=refractive.m_w_0C[tmatrix_aux.wl_W]).qb() * np.pi * (i / 2.) ** 2 for w, i in
             enumerate(ds)]
    df_scatter = pd.DataFrame(
        {'T_mat_Ku': tmat_ku, 'T_mat_Ka': tmat_ka, 'T_mat_W': tmat_w, 'Mie_Ku': mie_ku, 'Mie_Ka': mie_ka,
         'Mie_W': mie_w}, index=ds)
    path_db = f'{path_data}/db'
    str_db = f"sqlite:///{path_db}/backscatter.sqlite"
    df_scatter.to_sql(f'{instrument}', con=str_db, if_exists='replace')
    return df_scatter


def ref_calc(xr_data, mie=False):
    ds = xr_data.diameter.values / 1e3
    try:
        path_db = f'{path_data}/db'
        make_dir(path_db)
        str_db = f"sqlite:///{path_db}/backscatter.sqlite"
        backscatter = pd.read_sql(f"{xr_data.attrs['instrument']}", con=str_db)
    except OperationalError:
        backscatter = bcksct(ds, xr_data.attrs['instrument'])

    ku_wvl = c / 14e9 * 1000
    ka_wvl = c / 35e9 * 1000
    w_wvl = c / 95e9 * 1000
    bcks = xr.Dataset.from_dataframe(backscatter).rename_dims({'index': 'diameter'}).rename({'index': 'diameter'})

    if mie:
        z_ku = (ku_wvl ** 4 / (np.pi ** 5 * 0.93)) * bcks.Mie_Ku * xr_data.psd * 1e6 * xr_data.d_d
        z_ka = (ka_wvl ** 4 / (np.pi ** 5 * 0.93)) * bcks.Mie_Ka * xr_data.psd * 1e6 * xr_data.d_d
        z_w = (w_wvl ** 4 / (np.pi ** 5 * 0.93)) * bcks.Mie_W * xr_data.psd * 1e6 * xr_data.d_d
    else:
        z_ku = (ku_wvl ** 4 / (np.pi ** 5 * 0.93)) * bcks.T_mat_Ku * xr_data.psd * 1e6 * xr_data.d_d / 1e3
        z_ka = (ka_wvl ** 4 / (np.pi ** 5 * 0.93)) * bcks.T_mat_Ka * xr_data.psd * 1e6 * xr_data.d_d / 1e3
        z_w = (w_wvl ** 4 / (np.pi ** 5 * 0.93)) * bcks.T_mat_W * xr_data.psd * 1e6 * xr_data.d_d / 1e3
        
    ref = z_ka.to_dataset(name='z_ka')
    ref['z_ku'] = z_ku
    ref['z_w'] = z_w
    return ref

def pds_parameters(xr_data):
    """
    Compute the psd parameters
    :param xr_data: partice size distribution in # L-1 um-1
    :return: list with lwc, dm, nw, z, and r
    """
    lwc = (np.pi / (6 * 1000.)) * (xr_data.psd * 1e6) * (xr_data.diameter * 1e-3) ** 3 * (xr_data.d_d * 1e-3)
    m4 = (xr_data.psd * 1e6) * (xr_data.diameter * 1e-3) ** 4 * xr_data.d_d * 1e-3
    m3 = (xr_data.psd * 1e6) * (xr_data.diameter * 1e-3) ** 3 * xr_data.d_d * 1e-3
    dm = m4.sum('diameter') / m3.sum('diameter')
    z = xr_data.psd * 1e6 * (xr_data.diameter * 1e-3) ** 6 * xr_data.d_d
    nw = 1e3 * (4 ** 4 / np.pi) * (lwc.sum('diameter') / dm ** 4)
    params = lwc.to_dataset(name='lwc')
    params['dm'] = dm
    params['z'] = z
    params['nw'] = nw
    return params

In [ ]:
ref = ref_calc(xr_2ds.sel(time=slice('2019-09-07 02:31:30', '2019-09-07 02:34:10')))

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12,8), dpi=300, sharex=True)

cbar = ax1.pcolormesh(data_1.time.values, 
                      data_1.diameter / 1e3, 
                      np.log10(data_1.psd.T.values * 1e6), 
                      vmin=0, vmax=10, cmap=my_cmap)

ax1.set_yscale('log')
# ax11.plot(lwc_filt.time, lwc_filt.sum('diameter'), c='k', lw=0.5)
add_colorbar_outside(cbar, ax1, label='Log10(N(D)) (Log10(mm-1 m-3))')

cbar2 = ax2.pcolormesh(lwc_1.time.values, 
                       lwc_1.diameter / 1e3, 
                       lwc_1.T.values, 
                       vmin=0, vmax=1, cmap=my_cmap)
ax22 = ax2.twinx()
ax22.plot(lwc_filt.time, lwc_filt.sum('diameter'), c='k', ls='--', lw=0.8, label="LWC 2DS")
ax22.plot(lwc_nev.time, lwc_nev, c='r', ls='--', lw=0.8, label='Nevzorov')

# cbar3 = ax3.pcolormesh(lwc_1.time.values, 
#                        lwc_1.diameter / 1e3, 
#                        lwc_1.T.values, 
#                        vmin=0, vmax=1, cmap=my_cmap)

ax33 = ax3.twinx()
ax33.plot(ref.time, 10 * np.log10(ref.z_ka.sum('diameter')), label='Ka')
ax33.plot(ref.time, 10 * np.log10(ref.z_ku.sum('diameter')), label='Ku')
ax33.plot(ref.time, 10 * np.log10(ref.z_w.sum('diameter')), label='W')
ax3.set_yscale('log')

ax2.set_yscale('log')
ax2.set_ylim(0.01, 10)
add_colorbar_outside(cbar2, ax2, 'LWC (g m-3 mm)')
ax1.set_ylabel('Diameter (mm)')
ax2.set_ylabel('Diameter (mm)')
ax3.set_xlabel('Time (UTC)')
ax22.set_ylabel('LWC (g m-3)')
ax22.legend()
ax11.set_ylabel('LWC (g m-3)')
ax33.set_ylabel('Reflectivity (dBZ)')
ax1.set_ylim(0.002, 10)
ax2.set_ylim(0.002, 10)
ax33.set_ylim(-10, 50)
ax33.legend()
title = f"{pd.to_datetime(data_1_filt.time.values[0]): %Y-%m-%d} UTC - {data_1_filt.attrs['aircraft']}"
fig.suptitle(title, fontsize=14, fontweight='bold', y=0.99)
path_save = f"{path_data}/results/qc_cloud_probes/{xr_2ds.attrs['aircraft']}/{xr_2ds.attrs['instrument']}"
make_dir(path_save)
# fig.savefig(f"{path_save}/{pd.to_datetime(data_1_filt.time.values[0]): %Y-%m-%d}_to_{pd.to_datetime(data_1_filt.time.values[-1]): %Y-%m-%d}_filt.jpg")
# fig.tight_layout()

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(12,10), dpi=300, sharex=True)

cbar = ax1.pcolormesh(ref.time.values, 
                      ref.diameter / 1e3, 
                      10 * np.log10(ref.z_ka.values), 
                      vmin=-30, vmax=10, cmap=my_cmap)

ax1.set_yscale('log')
# ax11.plot(lwc_filt.time, lwc_filt.sum('diameter'), c='k', lw=0.5)
add_colorbar_outside(cbar, ax1, label='Reflectivity \n (dBZ)')

cbar2 = ax2.pcolormesh(ref.time.values, 
                      ref.diameter / 1e3, 
                      10 * np.log10(ref.z_ku.values), 
                      vmin=-30, vmax=10, cmap=my_cmap)

ax2.set_yscale('log')
# ax11.plot(lwc_filt.time, lwc_filt.sum('diameter'), c='k', lw=0.5)
add_colorbar_outside(cbar2, ax2, label='Reflectivity \n (dBZ)')

cbar3 = ax3.pcolormesh(ref.time.values, 
                      ref.diameter / 1e3, 
                      10 * np.log10(ref.z_w.values), 
                      vmin=-30, vmax=10, cmap=my_cmap)

add_colorbar_outside(cbar3, ax3, label='Reflectivity \n (dBZ)')


cbar4 = ax4.pcolormesh(lwc_1.time.values, 
                      lwc_1.diameter / 1e3, 
                      lwc_1.T.values, 
                      vmin=0, vmax=1, cmap=my_cmap)

add_colorbar_outside(cbar4, ax4, label='LWC (gr m-3)')

ax2.set_yscale('log')
ax3.set_yscale('log')
ax4.set_yscale('log')

ax3.set_ylim(0.01, 10)
ax4.set_ylim(0.01, 10)
ax2.set_ylim(0.01, 10)
ax1.set_ylabel('Diameter (mm)')
ax2.set_ylabel('Diameter (mm)')
ax3.set_xlabel('Time (UTC)')
ax22.set_ylabel('LWC (g m-3)')
ax22.legend()
ax11.set_ylabel('LWC (g m-3)')
ax33.set_ylabel('Reflectivity (dBZ)')
ax1.set_ylim(0.002, 10)
ax2.set_ylim(0.002, 10)
ax33.set_ylim(-10, 50)
ax33.legend()
title = f"{pd.to_datetime(data_1_filt.time.values[0]): %Y-%m-%d} UTC - {data_1_filt.attrs['aircraft']}"
fig.suptitle(title, fontsize=14, fontweight='bold', y=0.99)
path_save = f"{path_data}/results/qc_cloud_probes/{xr_2ds.attrs['aircraft']}/{xr_2ds.attrs['instrument']}"
make_dir(path_save)
# fig.savefig(f"{path_save}/{pd.to_datetime(data_1_filt.time.values[0]): %Y-%m-%d}_to_{pd.to_datetime(data_1_filt.time.values[-1]): %Y-%m-%d}_filt.jpg")
# fig.tight_layout()

In [ ]:
parms = pds_parameters(xr_2ds.sel(time=slice('2019-09-07 02:31:30', '2019-09-07 02:34:10')))

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12,8), dpi=300, sharex=True)
cbar1 = ax1.pcolormesh(parms.time, 
                       parms.diameter / 1e3, 
                       parms.lwc.T.values, 
                       vmin=0, vmax=1, cmap=my_cmap)

ax2.plot(parms.time, parms.dm, label='Dm')
ax3.plot(parms.time, np.log10(parms.nw), label='Nw')


add_colorbar_outside(cbar, ax1, 'LWC (gr m-3)')
ax1.set_yscale('log')
ax1.set_ylim(0.01, 10)

In [ ]:
fig, ax = plt.subplots()
cbar = ax.scatter(parms.dm, np.log10(parms.nw), c=parms.lwc.sum('diameter'))
add_colorbar_outside(cbar, ax, 'LWC (gr m-3)')
ax.set_xlabel('log10(Nw)')
ax.set_ylabel('Dm')

In [ ]:
data_res = xr_2ds.sel(time=slice('2019-09-07 07:32', '2019-09-07 07:35'))

In [ ]:
parms_res = pds_parameters(data_res)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12,8), dpi=300, sharex=True)
cbar1 = ax1.pcolormesh(parms_res.time, 
                       parms_res.diameter / 1e3, 
                       parms_res.lwc.T.values, 
                       vmin=0, vmax=1, cmap=my_cmap)
ax11 = ax1.twinx()
ax11.plot(parms_res.time, parms_res.lwc.sum('diameter'))

ax2.plot(parms_res.time, parms_res.dm, label='Dm')
ax3.plot(parms_res.time, np.log10(parms_res.nw), label='Nw')


add_colorbar_outside(cbar, ax1, 'LWC (gr m-3)')
ax1.set_yscale('log')
ax1.set_ylim(0.01, 10)

In [ ]:
df = xr_2ds.psd.sel(time=slice('2019-09-07 07:32', '2019-09-07 07:35')).to_dataframe()

In [ ]:
for i in xr_2ds.sel(time=slice('2019-09-07 02:31:50', '2019-09-07 02:32:15')).time:
    fig, ax1 = plt.subplots()
    
    y = xr_fssp.psd.sel(time=i) * 1e6
    y = y.where(y > 0, np.nan)

    ax1.step(x=xr_fssp.diameter * 1e-3, y=y, where='mid', c='r', lw=1, label= 'FFSP')
    
    y = xr_fcdp.psd.sel(time=i) * 1e6
    y = y.where(y > 0, np.nan)

    ax1.step(x=xr_fcdp.diameter * 1e-3, y=y, where='mid', c='brown', lw=1, label= 'FCDP')
    
    y = xr_2ds.psd.sel(time=i) * 1e6
    y = y.where(y > 0, np.nan)

    ax1.step(x=xr_2ds.diameter * 1e-3, y=y, where='mid', label='2DS', c='k')

    y = xr_hvps.psd.sel(time=i) * 1e6
    y = y.where(y > 0, np.nan)
    ax1.step(x=xr_hvps.diameter* 1e-3, y=y, where='mid', label='HVPS', c='b')
    
    ax1.vlines(x=xr_fssp.diameter.min() * 1e-3, ymin=1e0, ymax=1e12, colors='C0', linestyles='dashed')
    ax1.vlines(x=xr_fssp.diameter.max() * 1e-3, ymin=1e0, ymax=1e12, colors='C0', linestyles='dashed')
    
    ax1.vlines(x=xr_2ds.diameter.min() * 1e-3, ymin=1e0, ymax=1e12, colors='C1', linestyles='dashed')
    ax1.vlines(x=xr_2ds.diameter.max() * 1e-3, ymin=1e0, ymax=1e12, colors='C1', linestyles='dashed')
    
    ax1.vlines(x=xr_hvps.diameter.min() * 1e-3, ymin=1e0, ymax=1e12, colors='C2', linestyles='dashed')
    ax1.vlines(x=xr_hvps.diameter.max() * 1e-3, ymin=1e0, ymax=1e12, colors='C2', linestyles='dashed')
    
    ax1.set_yscale('log')
    ax1.set_xscale('log')
    ax1.legend()

    ax1.set_xlabel('Particle size (mm)')
    title = f"{pd.to_datetime(i.values): %Y-%m-%d %X} UTC - {aircraft}"
    fig.suptitle(title, fontsize=14, fontweight='bold', y=0.99)
#     ax1.set_ylabel('Concentration (# m-3 m-1)')
    ax1.grid(which='both')
    ax.grid(which='both')
    plt.show()